<a href="https://colab.research.google.com/github/jxb5778/topic_modeling/blob/master/LDA_Topic_Model_20_News_Group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

import nltk
from nltk import word_tokenize       
from nltk.corpus import stopwords   
from nltk.stem import WordNetLemmatizer 

import re


In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 552kB 16.5MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=d81f3519d23beaf66f896f56fbb2dff49e453112359720385d9b13e6cc8ad55c
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=fe15f59da48318c56d78fbc95048c957af14e4ae959cb512ed97bb79a8fc44b4
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [ ]:
import pyLDAvis
import pyLDAvis.gensim

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from sklearn.datasets import fetch_20newsgroups


data, _ = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'),
                             return_X_y=True)

In [ ]:
print(len(data))

11314


In [ ]:
print(data[5])

 
 
I read somewhere, I think in Morton Smith's _Jesus the Magician_, that
old Lazarus wasn't dead, but going in the tomb was part of an initiation
rite for a magi-cult, of which Jesus was also a part.   It appears that
a 3-day stay was normal.   I wonder .... ?


In [ ]:
class LemmaTokenizer(object):
    def __init__(self, token_pattern=r"(?u)\b\w\w+\b", stop='english'):
        self.wnl = WordNetLemmatizer()
        self.token_pattern = re.compile(token_pattern)
        self.stop = set(stopwords.words(stop))
        
    def __call__(self, doc):
        return [
                self.wnl.lemmatize(t) for t in word_tokenize(doc) 
                if len(t) > 2 
                and re.match("[a-z].*",t) 
                and re.match(self.token_pattern, t) 
                and t not in self.stop
                ]

In [ ]:
tokenizer = LemmaTokenizer()

In [ ]:
data_tokenized = [tokenizer(doc) for doc in data]

In [ ]:
bigram = gensim.models.Phrases(data_tokenized, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [ ]:
texts = [bigram_mod[doc] for doc in data_tokenized]

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(texts)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 3), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)]]


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                      id2word=id2word,
                                      num_topics=20, 
                                      random_state=100,
                                      chunksize=100,
                                      passes=10,
                                      alpha='asymmetric',
                                      eta='auto',
                                      per_word_topics=True)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.029*"key" + 0.016*"use" + 0.012*"system" + 0.010*"public" + 0.008*"chip" + 0.008*"information" + 0.007*"stream" + 0.007*"security" + 0.007*"used" + 0.006*"user"'), (1, '0.033*"drive" + 0.030*"system" + 0.021*"disk" + 0.016*"card" + 0.013*"use" + 0.011*"monitor" + 0.009*"mode" + 0.009*"data" + 0.008*"memory" + 0.008*"standard"'), (2, '0.018*"said" + 0.014*"child" + 0.012*"people" + 0.012*"one" + 0.010*"say" + 0.009*"time" + 0.009*"know" + 0.009*"could" + 0.009*"woman" + 0.008*"day"'), (3, '0.031*"perfect" + 0.010*"immune_system" + 0.009*"club" + 0.009*"food" + 0.009*"percentage" + 0.008*"soon" + 0.007*"hockey" + 0.007*"music" + 0.007*"illness" + 0.007*"eye"'), (4, '0.016*"would" + 0.013*"people" + 0.012*"one" + 0.011*"say" + 0.011*"think" + 0.009*"know" + 0.009*"see" + 0.009*"thing" + 0.008*"believe" + 0.008*"like"'), (5, '0.012*"war" + 0.010*"stand" + 0.010*"church" + 0.009*"crime" + 0.009*"speaking" + 0.009*"attack" + 0.009*"peace" + 0.008*"murder" + 0.007*"criminal" + 0.006*"

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us


Perplexity:  -9.44984125222119

Coherence Score:  -4.477741592688124


In [ ]:
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.227493  0.018794       1        1  20.212593
7      0.188234  0.121566       2        1   8.787473
8      0.185182 -0.106298       3        1   8.502315
2      0.157118  0.167123       4        1   7.250924
17     0.028586 -0.016371       5        1   6.962080
19     0.010229 -0.284073       6        1   6.117409
15     0.180452  0.003759       7        1   5.871411
0      0.011750 -0.165500       8        1   5.770492
13     0.117854  0.100188       9        1   5.264302
1      0.029548 -0.127737      10        1   4.666013
11    -0.067723 -0.126094      11        1   3.422567
5     -0.072084  0.073613      12        1   3.247719
3     -0.079423  0.089986      13        1   2.455147
14    -0.097406 -0.003867      14        1   2.103464
10    -0.120758  0.043522      15        1   1.834850
6     -0.110499  0.068582      16        1   1.658105
18    -0.136042  0.063943      17        1   1.638571
12    -0.140508  0.014309      18        1   1.538122
9     -0.156620  0.011049      19        1   1.362139
16    -0.155384  0.053505      20        1   1.334299, topic_info=          Term         Freq        Total Category  logprob  loglift
242       file  2324.000000  2324.000000  Default  30.0000  30.0000
131      drive  1217.000000  1217.000000  Default  29.0000  29.0000
142        key  1286.000000  1286.000000  Default  28.0000  28.0000
2885      game  1444.000000  1444.000000  Default  27.0000  27.0000
277    program  2112.000000  2112.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
36997    goose    45.871754    46.224659  Topic20  -5.4250   4.3091
2795   payload    48.740879    49.124725  Topic20  -5.3643   4.3089
2518       lab    48.809090    49.195156  Topic20  -5.3629   4.3089
4338    placed    73.605927    86.654480  Topic20  -4.9521   4.1536
6977    spring    73.743385    88.681152  Topic20  -4.9502   4.1323

[1065 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
5641     12  0.999047     absolute
5382     13  0.991677  accelerator
414       3  0.368929       access
414       8  0.627180       access
2422      9  1.000452     accident
...     ...       ...          ...
181      19  0.002114         year
181      20  0.001057         year
5720      9  0.992115       yellow
7729     16  0.999750         zero
2384     12  1.000252         zone

[3102 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 9, 3, 18, 20, 16, 1, 14, 2, 12, 6, 4, 15, 11, 7, 19, 13, 10, 17])

In [ ]:

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3, coherence='c_v'):
    """
    Compute log_perplexity and coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    id2word: Gensim id2word
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    log_perplexity_values: Log perplexity values corresponding to the LDA model with respective number of topics
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    log_perplexity_values = list()
    coherence_values = list()
    model_list = list()

    for num_topics in range(start, limit, step):
        model = gensim.models.LdaMulticore(corpus=corpus,
                                      id2word=dictionary,
                                      num_topics=20, 
                                      random_state=100,
                                      chunksize=100,
                                      passes=10,
                                      alpha='asymmetric',
                                      eta='auto',
                                      per_word_topics=True)
        model_list.append(model)

        log_perplexity = model.log_perplexity(corpus)
        log_perplexity_values.append(log_perplexity)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, log_perplexity_values, coherence_values


In [ ]:
# Will take a long time
model_list, log_perplexity_values, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=2, limit=40, step=6)

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

In [ ]:
import pandas as pd


df = pd.DataFrame({
    'num_topics': range(2, 40, 6),
    
})


In [ ]:
import plotly.express as px


fig = px.scatter(df, x="sepal_width", y="sepal_length")
fig.show()